# Знакомство с объектами библиотеки Pandas

Библиотека Pandas состоит из 3-х фундаментальных структур
- Series
- DataFrame
- Index

In [1]:
import numpy as np
import pandas as pd
pd.__version__

'0.25.1'

## Объект Series библиотеки Pandas

Объект Series библиотеки Pandas - одномерный массив индексированных данных. Его можно создать из списка или массива следующим образом:

In [2]:
data = pd.Series([0.25, 0.5, 0.75, 1.0])
data

0    0.25
1    0.50
2    0.75
3    1.00
dtype: float64

Как мы видели из предыдущего результата, объект Series служит адаптером как для последовательности значений, так и последовательности индексов, к которым можно получить доступ посредством атрибутов **values** и **index**. Атрибут values представляет представляет собой уже знакомый нам массив NumPy:

In [3]:
data.values

array([0.25, 0.5 , 0.75, 1.  ])

**index** - массивоподобный объект типа pd.Index, который мы рассмотрим подробнее далее:

In [4]:
data.index

RangeIndex(start=0, stop=4, step=1)

### Объект Series как обобщенный массив NumPy
Может показаться, что объект Series и одномерный массив библиотеки NumPy взаимозаменяемы. Основное различие между ними - индекс. В то время как индекс массива NumPy, целочисленный и описывается неявно, индекс объекта Series библиотеки Pandas описывается явно и связывается со значениями.

Явное описание индекса расширяет возможности объекта Series. Такой индекс не должен быть целым числом, а может состоять из значений любого нужного типа. Например при желании мы можем использовать в качестве индекса строковые значения: 

In [5]:
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 ['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [6]:
data.index

Index(['a', 'b', 'c', 'd'], dtype='object')

При этом доступ к элементам работает обычным образом:

In [7]:
data['b']

0.5

### Объект Series как специализированный словарь

Объект Series библиотеки Pandas можно рассматривать как специализированную разновидность словаря языка Python. Словарь - структура, задающая соответствие произвольных ключей набору произвольных значений, а объект Series - структура, задающая соответствие типизированных ключей набору типизированных значений. Типизация важна: точно так же, как соответствующий типу специализированный код для массива библиотеки NumPy при выполнении определенных операций делает его эффективнее, чем стандартный список Python, информация о типе в объекте Series библиотеки Pandas делает его намного более эффективным для определенных операций, чем словари Python.

Можно сделать аналогию "объект Series - словарь" еще более наглядной, сконструировав объект Series непосредственно из словаря Python:

In [8]:
population_dict = {'California': 38332521,
                   'Texas': 26448193,
                   'New York': 19651127,
                   'Florida': 19552860,
                   'Illinois': 12882135}

population = pd.Series(population_dict)
population

California    38332521
Texas         26448193
New York      19651127
Florida       19552860
Illinois      12882135
dtype: int64

По умолчанию при этом будет создан объект Series с полученным из отсортированных ключей, индексом. Следовательно для него возможен обычный доступ к элементам, такой же, как для словаря: 

In [9]:
population['California']

38332521

Однако в отличии от словаря, объект Series поддерживает характерные для массивов операции, такие как срезы:

In [10]:
population['California': 'Illinois']

California    38332521
Texas         26448193
New York      19651127
Florida       19552860
Illinois      12882135
dtype: int64

### Создание объектов Series

Мы уже изучили несколько способов создания объектов Series библиотеки Pandas с нуля. Все они представляют собой различные варианты следующего синтаксиса:

```Python
pd.Series(data, index=index)
```

где **index** - необязательный аргумент, а **data** может быть одной из множества сущностей (например списком, или массивом NumPy). В этом случае index по умолчанию будет целочисленной последовательностью:  

In [11]:
pd.Series([2, 4, 6])

0    2
1    4
2    6
dtype: int64

Аргумент **data** может быть скалярным значением, которое будет повторено нужное количество раз для заполнения заданного индекса:

In [12]:
pd.Series(5, index=[100, 200, 300])

100    5
200    5
300    5
dtype: int64

Аргумент **data** может быть словарем, в котором **index** по умолчанию является отсортированными ключами этого словаря: 

In [13]:
pd.Series({2: 'a', 1: 'b', 3: 'c'})

2    a
1    b
3    c
dtype: object

Однако пример показывает что ключи неотсортированы. Впрочем похуй.

В каждом случае индекс можно указать вручную, если необходимо получить другой результат:

In [14]:
pd.Series({2: 'b', 1: 'a', 3: 'c'}, index=[2, 3])

2    b
3    c
dtype: object

Обратим внимание, что объект Series заполняется только заданными явным образом ключами.

## Объект DataFrame библиотеки Pandas

Следующая базовая структура библиотеки Pandas - объект DataFrame. Как и объект Series, объект DataFrame можно рассматривать или как специализированную версию словаря Python. Рассмотрим оба варианта.

### DataFrame как обобщенный массив NumPy

Если объект Series - аналог одномерного массива с гибкими индексами, объект DataFrame - аналог двумерного массива с гибкими индексами строк и гибкими именами столбцов. Аналогично тому, что двумерный массив можно рассматривать как упорядоченную последовательность выровненных столбцов, объект DataFrame можно рассматривать как упорядоченную последовательность объектов Series. Под "выравненными" имеется в виду то, что они используют один и тот же индекс.

Чтобы продемонстрировать это, сначала создадим новый объект Series, содержащий площадь каждого из пяти упомянутых ранее штатов:

In [15]:
population_dict = {'California': 38332521,
                   'Texas': 26448193,
                   'New York': 19651127,
                   'Florida': 19552860,
                   'Illinois': 12882135}

population = pd.Series(population_dict)

area_dict = {'California': 423967,
             'Texas': 695662,
             'New York': 141297,
             'Florida': 170312,
             'Illinois': 149995}

area = pd.Series(area_dict)
area

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
dtype: int64

Воспользовавшись объектом *population* класса Series, сконструируем на основе словаря единый двумерный объект, содержащий всю эту информацию:

In [16]:
states = pd.DataFrame({'population': population,
                       'area': area})

states

,population,area
California,38332521,423967
Texas,26448193,695662
New York,19651127,141297
Florida,19552860,170312
Illinois,12882135,149995


Аналогично объекту Series у объекта DataFrame есть атрибут **index**, обеспечивающий доступ к меткам индекса:

In [17]:
states.index

Index(['California', 'Texas', 'New York', 'Florida', 'Illinois'], dtype='object')

Помимо этого, у объекта DataFrame есть атрибут **columns**, представляющий собой содержащий метки столбцов объект **Index**:

In [18]:
states.columns

Index(['population', 'area'], dtype='object')

Таким образом, объект DataFrame можно рассматривать как обобщение двумерного массива NumPy, где как у строк, так и у столбцов есть обобщенные индексы для доступа к данным.

### Объект DataFrame как специализированный словарь

DataFrame можно рассматривать как специализированный словарь. Если словарь задает соответствие ключей значениям, то DataFrame задает соответствие имени столбца объекту Series с данными этого столбца. Например, запрос данных по атрибуту 'area' приведет к тому, что будет возвращен объект Series, содержащий уже виденные нами ранее площади штатов:

In [19]:
states['area']

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

Обратим внимание на возможный источник путаницы: в двумерном массиве NumPy строка:
```Python
data[0]
```
возвращает первую строку. По этой причине объекты DataFrame лучше рассматривать как обобщенные словари, а не обобщенные массивы, хотя обе точки зрения имеют право на жизнь.

### Создание объектов DataFrame

Существует множество способов создания объектов DataFrame. Вот несколько примеров:

**Из одного объекта Series**. Объект DataFrame - набор объектов Series. DataFrame, состоящий из одного столбца, можно создать на основе одного объекта Series:

In [20]:
pd.DataFrame(population, columns=['population'])

,population
California,38332521
Texas,26448193
New York,19651127
Florida,19552860
Illinois,12882135


**Из списка словарей**. Любой список словарей можно преобразовать в объект DataFrame. Мы воспользуемся простым списковым включением для создания данных:

In [21]:
data = [{'a': i, 'b': 2 * i} for i in range(3)]

pd.DataFrame(data)

,a,b
0,0,0
1,1,2
2,2,4


Даже если некоторые ключи в словаре отсутствуют, библиотека Pandas просто заполнит их значениями **NaN** (т.е. Not a number - "не является числом"):

In [22]:
pd.DataFrame([{'a': 1, 'b': 2}, {'b': 3, 'c': 4}])

,a,b,c
0,1.0,2,NaN
1,NaN,3,4.0


**Из словаря объектов Series**. Объект DataFrame так же можно создать на основе словаря объектов Series:

In [23]:
pd.DataFrame({'population': population, 'area': area})

,population,area
California,38332521,423967
Texas,26448193,695662
New York,19651127,141297
Florida,19552860,170312
Illinois,12882135,149995


**Из двумерного массива NumPy**. Если у нас есть двумерный массив данных, мы можем создать объект DataFrame c любыми заданными именами столбцов и индексов. Для каждого из пропущенных значений будет использоваться целочисленный индекс:

In [24]:
pd.DataFrame(np.random.rand(3, 2),
             columns=['foo', 'bar'],
             index=['a', 'b', 'c'])

,foo,bar
a,0.117465,0.141997
b,0.729084,0.938378
c,0.339265,0.338874


**Из структурированного массива NumPy**. Объект DataFrame библиотеки Pandas ведет себя во многом аналогично структурированному массиву и может быть создан непосредственно из него:

In [25]:
A = np.zeros(3, dtype=[('A', 'i8'), ('B', 'f8')])
A

array([(0, 0.), (0, 0.), (0, 0.)], dtype=[('A', '<i8'), ('B', '<f8')])

In [26]:
pd.DataFrame(A)

,A,B
0,0,0.0
1,0,0.0
2,0,0.0


## Объект Index библиотеки Pandas

Объект Index можно рассматривать или как неизменяемый массив, или как упорядоченное множество. Из этих способов его представления следуют некоторые интересные возможности операций над объектами Index. В качестве простого примера создадим Index из списка чисел:

In [27]:
ind = pd.Index([2, 3, 5, 7, 11])
ind

Int64Index([2, 3, 5, 7, 11], dtype='int64')

### Объект Index как неизменяемый массив

Объект Index во многом ведет себя аналогично массиву. Например, для извлечения из него значений или срезов можно использовать стандартную нотацию индексации языка Python:

In [28]:
ind[1]

3

In [29]:
ind[::2]

Int64Index([2, 5, 11], dtype='int64')

У объектов Index есть много атрибутов, знакомых нам по массивам NumPy:

In [30]:
print(ind.size, ind.shape, ind.ndim, ind.dtype)

5 (5,) 1 int64


Одно из различий между объектами Index и массивами NumPy - неизменяемость индексов, т.е. их нельзя модифицировать стандартными средствами:

In [31]:
ind[1] = 0

TypeError: Index does not support mutable operations

Неизменяемость делает безопаснее совместное использование индексов несколькими объектами DataFrame и массивами, исключая возможность побочных эффектов в виде случайной модификации индекса по неосторожности.

### Index как упорядоченное множество

Объекты библиотеки Pandas спроектированы с прицелом на упрощение таких операций, как соединения наборов данных, зависящие от многих аспектов арифметики множеств. Объект Index следует большинству соглашений, используемых встроенной структурой данных **set** (множество) языка Python, так что объединения, персечения, разности и другие операции над множествами можно выполнять привычным образом: 

In [32]:
indA = pd.Index([1, 3, 5, 7, 9])
indB = pd.Index([2, 3, 5, 7, 11])

In [33]:
indA & indB # пересечение

Int64Index([3, 5, 7], dtype='int64')

In [34]:
indA | indB # объединение

Int64Index([1, 2, 3, 5, 7, 9, 11], dtype='int64')

In [35]:
indA ^ indB # симметричная разность

Int64Index([1, 2, 9, 11], dtype='int64')

Эти операции можно выполнять так же методами объектов, например:

In [36]:
indA.intersection(indB)

Int64Index([3, 5, 7], dtype='int64')

## Индексация и выборка данных

### Выборка данных из объекта Series

Объект Series во многом ведет себя подобно одномерному массиву библиотеки NumPy и стандартному словарю языка Python. Это поможет нам лучше понимать паттерны индексации и выборки данных из массивов.

### Объект Series как словарь

Объект Series задает соответствие набора ключей набору значений аналогично словарю:

In [37]:
import pandas as pd

data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])

data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [38]:
data['b']

0.5

Для просмотра ключей/индексов и значений выражения можно также использовать методы языка Python, аналогичные таковым для словарей:

In [39]:
'a' in data

True

In [40]:
data.keys()

Index(['a', 'b', 'c', 'd'], dtype='object')

In [41]:
list(data.items())

[('a', 0.25), ('b', 0.5), ('c', 0.75), ('d', 1.0)]

Объекты Series можно модифицировать с помощью синтаксиса, подобного синтаксису для словарей. Аналогично расширению словаря путем присваивания значения для нового ключа можно расширять объект Series, присвоив значение для нового значения индекса:

In [42]:
data['e'] = 1.25
data

a    0.25
b    0.50
c    0.75
d    1.00
e    1.25
dtype: float64

Такая легкая изменяемость объектов - удобная возможность: библиотека Pandas сама, незаметно для нас, принимает решения о размещении в памяти и необходимости копирования данных.

### Объект Series как одномерный массив

Объект Series, основываясь на интерфейсе, напоминающем словарь, предоставляет возможность выборки элементов с помощью тех же базовых механизмов, что и для массивов NumPy:
- срезы
- маскирование
- прихотливая индексация

In [43]:
# срез посредством явного индекса
data['a': 'c']

a    0.25
b    0.50
c    0.75
dtype: float64

In [44]:
# срез посредством неявного целочисленного индекса
data[0: 2]

a    0.25
b    0.50
dtype: float64

In [45]:
# маскирование
data[(data > 0.3) & (data < 0.8)]

b    0.50
c    0.75
dtype: float64

In [46]:
# Прихотливая индексация
data[['a', 'e']]

a    0.25
e    1.25
dtype: float64

Наибольшие затруднения среди них могут вызвать срезы. Обратите внимание, что при выполнении среза с помощью явного индекса *data['a': 'c']* значение соответствующее последнему индексу, включается в срез, а при срезе неявным индексом *data[0:2]* - не включается.

### Индексаторы: loc, iloc и ix

Подобные обозначения для срезов и индексации могут привести к путанице. Например, при наличии у объекта Series явного целочисленного индекса операция индексации *data[1]* будет использовать явные индексы, а операция среза *data[1:3]* - неявный индекс в стиле языка Python: 

In [47]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
data

1    a
3    b
5    c
dtype: object

In [48]:
# Использование явного индекса при индексации
data[1]

'a'

In [49]:
# Использование неявного индекса при срезе
data[1:3]

3    b
5    c
dtype: object

Из-за этой потенциальной путаницы в случае целочисленных индексов в библиотеке Pandas предусмотрены специальные атрибуты-*индексаторы*, позволяющие явным образом применять определенные схемы индексации. Они являются не функциональными методами, а именно атрибутами, предоставляющими для данных из объекта Series определенный интерфейс для выполнения срезов.

Во-первых, атрибут **loc** позволяет выполнить индексацию и срезы с использованием явного индекса:

In [50]:
data.loc[1]

'a'

In [51]:
data.loc[1:3]

1    a
3    b
dtype: object

Атрибут **iloc** дает возможность выполнить индексацию и срезы, применяя неявный индекс в стиле языка Python:

In [52]:
data.iloc[1]

'b'

In [53]:
data.iloc[1:3]

3    b
5    c
dtype: object

Третий атрибут-индексатор **ix** представляет собой гибрид первых двух и для объектов Series эквивалентен обычной индексации с помощью []. Назначение индексатора ix станет понятнее в контексте объектов DataFrame.

## Выборка данных из объекта DataFrame

Объект DataFrame во многом ведет себя аналогично двумерному или структурированному массиву, а так же словарю объектов Series с общим индексом. Эти аналогии следует иметь в виду во время изучения способов выборки данных из объекта.

### Объект DataFrame как словарь

Первая аналогия, которую мы будем обсуждать, - объект DataFrame как словарь схожих между собой объектов Series. Вернемся к примеру про площадь и численность населения штатов:

In [55]:
states_list = ['California',
               'Texas',
               'New York',
               'Florida',
               'Illinois']

area = pd.Series([423967, 695662, 141297, 170312, 149995], index=states_list)
pop = pd.Series([38332521, 26448193, 19651127, 19552860, 12882135], index=states_list)

data = pd.DataFrame({'area': area, 'pop': pop})
data

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


К отдельным объектам Series, составляющим столбцы объекта DataFrame, можно обращаться посредством такой же индексации, как и для словарей, по имени столбца:

In [56]:
data['area']

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

Можно добавить еще один столбец:

In [57]:
data['density'] = data['pop'] / data['area']
data

,area,pop,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


### Объект DataFrame как двумерный массив

Объект DataFrame можно рассматривать как двумерный массив с расширенными возможностями. Взглянем на исходный массив с помощью атрибута **values**:

In [58]:
data.values

array([[4.23967000e+05, 3.83325210e+07, 9.04139261e+01],
       [6.95662000e+05, 2.64481930e+07, 3.80187404e+01],
       [1.41297000e+05, 1.96511270e+07, 1.39076746e+02],
       [1.70312000e+05, 1.95528600e+07, 1.14806121e+02],
       [1.49995000e+05, 1.28821350e+07, 8.58837628e+01]])

Мы можем выполнить множество привычных для массивов действий над объектом DataFrame. Например, транспонировать весь DataFrame, поменяв местами строки и столбцы:

In [60]:
data.T

,California,Texas,New York,Florida,Illinois
area,4.239670e+05,6.956620e+05,1.412970e+05,1.703120e+05,1.499950e+05
pop,3.833252e+07,2.644819e+07,1.965113e+07,1.955286e+07,1.288214e+07
density,9.041393e+01,3.801874e+01,1.390767e+02,1.148061e+02,8.588376e+01


Однако, когда речь заходит об индексации объектов DataFrame, становится ясно, что словарная индексация мешает нам рассматривать их просто как массивы NumPy. В частности, указание отдельного индекса для массива означает доступ к строке:

In [61]:
data.values[0]

array([4.23967000e+05, 3.83325210e+07, 9.04139261e+01])

а указание отдельного "индекса" для объекта DataFrame - доступ к столбцу:

In [62]:
data['area']

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

Таким образом, нам необходим еще один тип синтаксиса для индексации, аналогичной по стилю индексации массивов. Библиотека Pandas применяет упомянутые ранее индексаторы:
- loc
- iloc
- ix

С помощью индексатора **iloc** можно индексировать исходный массив, как будто это массив NumPy (используя неявный синтаксис языка Python), но с сохранением результирующих данных меток объекта DataFrame для индекса и столбцов:

In [63]:
data

,area,pop,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


In [69]:
data.iloc[:3]

,area,pop,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746


In [73]:
data.iloc[:3,:2]

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127


In [77]:
data.loc[:'Florida', :'pop']

,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860


Индексатор **ix** позволяет комбинировать эти два подхода:

In [78]:
data.ix[:3, :'pop']

C:\Users\shiale\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """Entry point for launching an IPython kernel.
C:\Users\shiale\Anaconda3\lib\site-packages\pandas\core\indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)


,area,pop
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127


Как видим, что индексатор **ix** устарел. Впрочем это похуй т.к. я все равно не собирался его использовать.

Любой такой синтаксис индексации можно применять для задания или изменения значений. Это выполняется обычным, уже привычным по работе с библиотекой NumPy способом:

In [84]:
data.iloc[[0]]

,area,pop,density
California,423967,38332521,90.413926


In [85]:
data.iloc[0, 2] = 90
data.iloc[[0]]

,area,pop,density
California,423967,38332521,90.0


### Дополнительный синтаксис для индексации

Существует еще несколько вариантов синтаксиса для индексации, казалось бы, плохо согласующихся с обсуждавшимися ранее, но очень удобных на практике. Во-первых, если индексация относится к столбцам, срезы относятся к строкам:

In [88]:
data['New York':'Illinois']

,area,pop,density
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


При подобных срезах можно также ссылаться на строки по номеру, а не по индексу:

In [89]:
data[0:3]

,area,pop,density
California,423967,38332521,90.000000
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746


Непосредственные операции маскирования тоже интерпретируются построчно, а не по столбцам:

In [90]:
data

,area,pop,density
California,423967,38332521,90.000000
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


In [92]:
data[data['area'] > 150000]

,area,pop,density
California,423967,38332521,90.000000
Texas,695662,26448193,38.018740
Florida,170312,19552860,114.806121


In [93]:
data[data['density'] > 100]

,area,pop,density
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121


Эти два варианта обозначений синтаксически подобны таковым для массивов библиотеки NumPy, и они возможно, хоть и не вполне вписываются в шаблоны синтаксиса библиотеки Pandas, но весьма удобны на практике.

# Операции над данными в библиотеке Pandas

Одна из важнейших составляющих библиотеки NumPy - способность выполнять быстрые поэлементные операции - как простейшие арифметические (сложение, вычитание, умножение и т.д.), так и более сложные (тригонометрические, показательные и логорифмические функции и т.п.). Библиотека Pandas наследует от NumPy немалую часть этой функциональности, и ключ к ее использованию - универсальные функции.

Также библиотека Pandas включает несколько полезных трюков для унарных операций, например изменения знака и тригонометрических функций, при использовании ее универсальных функций в выводе будут *сохранены индекс и метки столбцов*, а для бинарных операций, например сложения и умножения, библиотека Pandas будет автоматически *выравнивать индексы* при передаче объектов универсальной функции. Это значит, что сохранение контекста данных и объединение данных из разных источников - две разные задачи, потенциально чреватые ошибками при работе с исходными массивами библиотеки NumPy, - становятся надежно защищенными от ошибок благодаря библиотеке Pandas. Кроме того, в библиотеке заданы операции между одномерными структурами объектов Series и двумерными структурами объектов DataFrame.

## Универсальные функции: сохранение индекса

В силу того что библиотека Pandas предназначена для работы с библиотекой NumPy, все универсальные функции библиотеки NumPy будут работать с объектами Series и DataFrame библиотеки Pandas. Начнем с описания простых объектов Series и DataFrame для демонстрации этого:

In [2]:
import pandas as pd
import numpy as np

In [3]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, 4))
ser

0    6
1    3
2    7
3    4
dtype: int32

In [4]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)), columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


Если применить универсальную функцию NumPy к любому из этих объектов, результатом будет *другой* объект библиотеки Pandas *с сохранением индексов*:

In [5]:
np.exp(ser)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

Или в случае немного более сложных вычислений:

In [6]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


## Универсальные функции: выравнивание индексов

При бинарных операциях над двумя объектами Series или DataFrame библиотека Pandas будет выравнивать индексы в процессе выполнения операции. Это очень удобно при работе с неполными данными.

### Выравнивание индексов в объектах Series

Допустим, мы объединили два различных источника данных, чтобы найти три штата США с наибольшей площадью и три штата США с наибольшим количеством населения.

In [7]:
area = pd.Series({'Alaska': 1723337,
                  'Texas': 695662,
                  'California': 423967}, name='area')
population = pd.Series({'California': 38332521,
                        'Texas': 26448193,
                        'New York': 19651127}, name='population')

Посмотрим, что получится, если разделить второй результат на первый для вычисления плотности населения:

In [8]:
population / area

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

Получившийся в итоге массив содержит объединение индексов двух исходных массивов, которое можно определить посредством стандартной арифметики множеств языка Python для этих индексов:

In [9]:
area.index | population.index

Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')

Ни один из относящихся к ним обоим элементов не содержит значения NaN ("нечисловое значение"), с помощью которого библиотека Pandas отмечает пропущенные данные. Аналогичным образом реализовано сопоставление индексов для всех встроенных арифметических выражений языка Python: все отсутствующие значения заполняются по умолчанию значением NaN:

In [11]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

Если использование значений NaN нежелательно, можно заменить заполняющее значение другим, воспользовавшись соответствующими методами объекта вместо операторов. Например, вызов метода **A.add(B)** эквивалентен вызову A + B, но предоставляет возможность по желанию задать явным образом значения заполнителей для любых потенциально отсутствующих элементов в объектах A или B:

In [12]:
A.add(B, fill_value=0)

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

### Выравнивание индексов в объектах DataFrame

При выполнении операций над объектами DataFrame происходит аналогичное выравнивание как для столбцов, так и для индексов:

In [13]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)), columns=['A', 'B'])
A

,A,B
0,1,11
1,5,1


In [14]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)), columns=['B', 'A', 'C'])
B

,B,A,C
0,4,0,9
1,5,8,0
2,9,2,6


In [15]:
A + B

,A,B,C
0,1.0,15.0,NaN
1,13.0,6.0,NaN
2,NaN,NaN,NaN


Обратите внимание, что индексы выравниваются правильно независимо от их расположения в двух объектах и индексы в полученном результате отсортированы. Как и в случае объектов Series, можно использовать соответвствующие арифметические методы объектов и передавать для использования вместо отсутствующих значений любое нужное значение fill_value. В следующем примере мы заполним отсутствующие значения средним значением всех элементов объекта A, которое вычислим, выстроив сначала значения объекта A в один столбец с помощью функции **stack()**:

In [17]:
fill = A.stack().mean()
A.add(B, fill_value=fill)

,A,B,C
0,1.0,15.0,13.5
1,13.0,6.0,4.5
2,6.5,13.5,10.5


Перечень операторов языка Python и эквивалентных им методов объектов библиотеки Pandas приведен в таблице 3.1 на странице 152

## Универсальные функции: выполнение операций между объектами DataFrame и Series

При выполнении операций между объектами DataFrame и Series выравнивание столбцов и индексов осуществляется аналогичным образом. Операции между объектами DataFrame и Series подобны операциям между двумерными и одномерными массивами библиотеки NumPy. Рассмотрим одну из часто встречающихся операций - вычисление разности двумерного массива и одной из его строк:

In [18]:
A = rng.randint(10, size=(3, 4))
A

array([[3, 8, 2, 4],
       [2, 6, 4, 8],
       [6, 1, 3, 8]])

In [19]:
A - A[0]

array([[ 0,  0,  0,  0],
       [-1, -2,  2,  4],
       [ 3, -7,  1,  4]])

В соответствии с правилами транслирования библиотеки NumPy, вычитание из двумерного массива одной из его строк выполняется построчно.

В библиотеке Pandas вычитание по умолчанию также происходит построчно:

In [21]:
df = pd.DataFrame(A, columns=list('QRST'))
df - df.iloc[0]

,Q,R,S,T
0,0,0,0,0
1,-1,-2,2,4
2,3,-7,1,4


Если же необходимо выполнить эту операцию по столбцами, можно воспользоваться методом **subtract()** указав параметр **axis**

In [22]:
df.subtract(df['R'], axis=0)

,Q,R,S,T
0,-5,0,-6,-4
1,-4,0,-2,2
2,5,0,2,7


Обратите внимание, что операции DataFrame/Series, аналогично обсуждавшимся ранее операциям, будут автоматически выполнять выравнивание индексов между двумя их элементами:

In [24]:
halfrow = df.iloc[0, ::2]
display(df)
display(halfrow)

,Q,R,S,T
0,3,8,2,4
1,2,6,4,8
2,6,1,3,8


Q    3
S    2
Name: 0, dtype: int32

In [25]:
df - halfrow

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,-1.0,NaN,2.0,NaN
2,3.0,NaN,1.0,NaN


Подобное сохранение и выравнивание индексов и столбцов означает, что операции над данными в библиотеке Pandas всегда сохраняют контекст данных, предотвращая возможные ошибки при работе с неоднородными и/или неправильно/неодинаково выравненными данными в исходных массивах NumPy.

## Обработка отсутствующих данных

Реальные данные редко бывают очищенными и однородными. Во многих интересных наборах данных некоторое количество данных отсутствует. Еще более затрудняет работу то, что в различных источниках данных отсутствующие данные могут быть помечены различным образом.

В этом разделе мы обсудим общие соображения, касающиеся отсутствующих данных, обсудим способы представления их библиотекой Pandas и продемонстрируем встроенные инструменты библиотеки Pandas для обработки отсутствующих данных в языке Python. Здесь и далее мы будем называть отсутствующие данные null, NaN или NA-значениями.